In [1]:
%pylab inline
from requests_html import HTMLSession
import pandas as pd


Populating the interactive namespace from numpy and matplotlib


In [2]:
url = 'http://apf.ststephens.edu/attendance_stku2018'
get_semester_url = 'http://apf.ststephens.edu/getsemesterbypro?programme_id={}'
get_month_url = 'http://apf.ststephens.edu/getmonthbysemepro?semester_id={}&programme_id={}'
session = HTMLSession()
r = session.get(url) 

In [3]:
course_options = r.html.find('option')[1:33]
programs = {  course.attrs['value'] : course.text
            for course in course_options }
programs

{'10': 'I YEAR SANSKRIT',
 '13': 'II YEAR B. A. PROGRAMME',
 '15': 'II YEAR B. A. HONOURS HISTORY',
 '16': 'II YEAR B. A. HONOURS SANSKRIT',
 '17': 'I YEAR HISTORY',
 '18': 'I YEAR ECONOMICS',
 '19': 'I YEAR MATHEMATICS',
 '20': 'I YEAR CHEMISTRY',
 '21': 'I YEAR PHYSICS',
 '22': 'II YEAR B. A. HONOURS ECONOMICS',
 '26': 'II YEAR B. SC. HONOURS CHEMISTRY',
 '28': 'II YEAR B. SC. HONOURS PHYSICS',
 '30': 'I YEAR B. SC. PROG. COMPUTER SCIENCE',
 '31': 'II YEAR B. SC. PROG. COMPUTER SCIENCE',
 '33': 'II YEAR B. SC. HONOURS MATHEMATICS',
 '35': 'I YEAR PHILOSOPHY',
 '36': 'II YEAR B. A. HONOURS PHILOSOPHY',
 '39': 'III YEAR B. A. HONOURS PHILOSOPHY',
 '40': 'III YEAR B. A. PROGRAMME',
 '42': 'III YEAR B. SC. HONOURS MATHEMATICS',
 '43': 'III YEAR B. A. HONOURS HISTORY',
 '44': 'III YEAR B. SC. HONOURS PHYSICS',
 '45': 'III YEAR B. A. HONOURS SANSKRIT',
 '47': 'III YEAR B. A. HONOURS ECONOMICS',
 '48': 'III YEAR B. SC. PROG. COMPUTER SCIENCE',
 '49': 'III YEAR B. SC. PROG. CHEMISTRY',
 '50'

In [4]:
p = session.get(get_semester_url.format(list(programs.keys())[0]))

In [5]:
sem_dict = p.json()
if sem_dict['status']:
    semesters = {}
    for entry in sem_dict['semester']:
        semesters[entry['semester_id']] = entry['name']

In [6]:
semesters

{'8': 'Semester I', '9': 'Semester II'}

In [7]:
q = session.get(get_month_url.format(list(semesters.keys())[0],list(programs.keys())[0]))

In [8]:
month_dict = q.json()
if month_dict['status'] and month_dict['month'] != []:
    months = {}
    for entry in month_dict['month']:
        months[entry['month_no']] = entry['name']
months    

{'07': 'July',
 '08': 'August',
 '09': 'September',
 '10': 'October',
 '11': 'November'}

## Getting attendance data

In [10]:
payload = {
    'programme' : list(programs.keys())[0],
    'semester' : list(semesters.keys())[0],
    'month' : list(months.keys())[0],
}
s = session.post(url, data=payload)


In [11]:
payload

{'month': '07', 'programme': '6', 'semester': '8'}

In [14]:
scraped_table = pd.read_html(s.html.html, header=1)[0]
scraped_table.drop(columns=scraped_table.columns.values.tolist()[-11:], inplace=True)
scraped_table['Month'] = months[payload['month']]
scraped_table['Semester'] = semesters[payload['semester']]

In [15]:
scraped_table.head()

,LD,LA,TH,TA,PH,PA,LD.1,LA.1,TH.1,TA.1,...,PA.5,LD.6,LA.6,TH.6,TA.6,PH.6,PA.6,LD.7,Month,Semester
0,Aditi Singh,0,0,0,0,0,0,3,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
1,Adreeta Chakraborty,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
2,Aishwarya Roy,7,6,1,0,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
3,Aiyana Aju Abraham,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
4,Ajay R Raj,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I


## TODO: Append all table and then pivot

In [17]:
papers = [
    paper.text 
    for paper in s.html.find(selector='th') 
    if 'colspan' in paper.attrs and paper.attrs['colspan'] == '6'
]
types = ['Lectures_Held', 'Lectures_Attended',
         'Tutorials_Held','Tutorials_Attended',
         'Practicals_Held', 'Practicals_Attended',]
headers = ['Name']
for paper in papers:
    for attendance_type in types:
        headers.append(paper+'_'+attendance_type)     
headers.append('Month') 
headers.append('Semester')

In [18]:
col_names = scraped_table.columns.values.tolist()
col_rename = dict(zip(col_names,headers))
scraped_table.rename(mapper=col_rename, axis=1,inplace=True)
scraped_table.head()

,Name,Indian Classical Literature_Lectures_Held,Indian Classical Literature_Lectures_Attended,Indian Classical Literature_Tutorials_Held,Indian Classical Literature_Tutorials_Attended,Indian Classical Literature_Practicals_Held,Indian Classical Literature_Practicals_Attended,European Classical Literature_Lectures_Held,European Classical Literature_Lectures_Attended,European Classical Literature_Tutorials_Held,...,"English Communications AECC IPhil, I Sans , I Engl_Practicals_Held","English Communications AECC IPhil, I Sans , I Engl_Practicals_Attended",Introduction to Physical Education in the Contemporary Context_Lectures_Held,Introduction to Physical Education in the Contemporary Context_Lectures_Attended,Introduction to Physical Education in the Contemporary Context_Tutorials_Held,Introduction to Physical Education in the Contemporary Context_Tutorials_Attended,Introduction to Physical Education in the Contemporary Context_Practicals_Held,Introduction to Physical Education in the Contemporary Context_Practicals_Attended,Month,Semester
0,Aditi Singh,0,0,0,0,0,0,3,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
1,Adreeta Chakraborty,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
2,Aishwarya Roy,7,6,1,0,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
3,Aiyana Aju Abraham,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I
4,Ajay R Raj,7,7,1,1,0,0,6,6,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,July,Semester I


In [19]:
scraped_melted = pd.melt(scraped_table,id_vars=['Name', 'Month', 'Semester'], var_name='Paper_Attendance')
paper_types = scraped_melted['Paper_Attendance'].str.split('_')
paper_names = paper_types.str.get(0)
class_type = paper_types.str.get(1)
attendance_type = paper_types.str.get(2)
scraped_melted['Paper Name'] = paper_names
scraped_melted['Class Type'] = class_type
scraped_melted['Attendance Type'] = attendance_type
scraped_melted.drop('Paper_Attendance', axis =1, inplace=True)

In [20]:
scraped_melted.head()

,Name,Month,Semester,value,Paper Name,Class Type,Attendance Type
0,Aditi Singh,July,Semester I,0.0,Indian Classical Literature,Lectures,Held
1,Adreeta Chakraborty,July,Semester I,7.0,Indian Classical Literature,Lectures,Held
2,Aishwarya Roy,July,Semester I,7.0,Indian Classical Literature,Lectures,Held
3,Aiyana Aju Abraham,July,Semester I,7.0,Indian Classical Literature,Lectures,Held
4,Ajay R Raj,July,Semester I,7.0,Indian Classical Literature,Lectures,Held


In [21]:
attend_tidy = scraped_melted.pivot_table(
    index=['Name', 'Semester','Paper Name', 'Month', 'Class Type'],
    columns='Attendance Type',
    values='value'
)
attend_tidy

Attendance Type                                                                                  Attended  \
Name             Semester   Paper Name                                         Month Class Type             
Aarzoo Jolly     Semester I Delhi Thought the Ages                             July  Lectures         NaN   
                                                                                     Practicals       NaN   
                                                                                     Tutorials        NaN   
                            English Communications AECC IPhil, I Sans , I Engl July  Lectures         6.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        0.0   
                            Ethics in the Public Domain                        July  Lectures         NaN   
                                                                                     Practicals       NaN   
                                                                                     Tutorials        NaN   
                            European Classical Literature                      July  Lectures         6.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        0.0   
                            Indian Classical Literature                        July  Lectures         7.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        1.0   
                            Introduction to Physical Education in the Conte... July  Lectures         NaN   
                                                                                     Practicals       NaN   
                                                                                     Tutorials        NaN   
                            Women Power and Politics GE I                      July  Lectures         8.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        0.0   
Aditi Singh      Semester I Delhi Thought the Ages                             July  Lectures         NaN   
                                                                                     Practicals       NaN   
                                                                                     Tutorials        NaN   
                            English Communications AECC IPhil, I Sans , I Engl July  Lectures         0.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        0.0   
                            Ethics in the Public Domain                        July  Lectures         NaN   
                                                                                     Practicals       NaN   
                                                                                     Tutorials        NaN   
...                                                                                                   ...   
Sheryl Alexander Semester I Indian Classical Literature                        July  Lectures         7.0   
                                                                                     Practicals       0.0   
                                                                                     Tutorials        0.0   
                            Introduction to Physical Education in the Conte.